<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/cs_whisper_small_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-82c5gzno
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-82c5gzno
  Resolved https://github.com/huggingface/transformers to commit f70db28322150dd986298cc1d1be8bc144cc1a88
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [23]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [24]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm


In [25]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "cs", split="test")
#column_name = "audio, normalized_text"

In [26]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1123
})

In [27]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [28]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 138MiB/s]


In [29]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [30]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [31]:
text

,0
0,tato norma posiluje pozici evropské agentury p...
1,dámy a pánové útok na židy je útok na nás samo...
2,zatímco se na půdě evropských institucí dohodl...
3,paní předsedající když hovoříme o humanitární ...
4,vážené poslankyně vážení poslanci zastupuji dn...
...,...
1118,ale vzhledem ke stárnutí populace a dalším výz...
1119,pane předsedající já myslím že můj přítel rich...
1120,erasmus.
1121,do budoucna je zřejmé a varují před tím analýz...


In [32]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [33]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,tato norma posiluje pozici evropské agentury p...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.00067138671875, -0.001...",16000
1,male,dámy a pánové útok na židy je útok na nás samo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.000213623046875, 0.0020...",16000
2,male,zatímco se na půdě evropských institucí dohodl...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001495361328125, -0.00152587890625, -0.0089...",16000
3,male,paní předsedající když hovoříme o humanitární ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00018310546875, -6.103515625e-05, 0.000762...",16000
4,male,vážené poslankyně vážení poslanci zastupuji dn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0001220703125, -0.000152587890625, -0.00015...",16000
...,...,...,...,...,...
1118,female,ale vzhledem ke stárnutí populace a dalším výz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00335693359375, -0.01141357421875, 0.00012...",16000
1119,male,pane předsedající já myslím že můj přítel rich...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00079345703125, -0.001068115234375, -0.001...",16000
1120,male,erasmus.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003448486328125, -0.00347900390625, -0.004...",16000
1121,male,do budoucna je zřejmé a varují před tím analýz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, 0.00469970703125, 0.00384...",16000


In [34]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,tato norma posiluje pozici evropské agentury p...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.00067138671875, -0.001...",16000
1,male,dámy a pánové útok na židy je útok na nás samo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.000213623046875, 0.0020...",16000
2,male,zatímco se na půdě evropských institucí dohodl...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001495361328125, -0.00152587890625, -0.0089...",16000
3,male,paní předsedající když hovoříme o humanitární ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00018310546875, -6.103515625e-05, 0.000762...",16000
4,male,vážené poslankyně vážení poslanci zastupuji dn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0001220703125, -0.000152587890625, -0.00015...",16000
...,...,...,...,...,...
1116,male,já když jsem slyšel pana komisaře jak si vymez...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001617431640625, -0.00146484375, 6.1035156...",16000
1119,male,pane předsedající já myslím že můj přítel rich...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00079345703125, -0.001068115234375, -0.001...",16000
1120,male,erasmus.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003448486328125, -0.00347900390625, -0.004...",16000
1121,male,do budoucna je zřejmé a varují před tím analýz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, 0.00469970703125, 0.00384...",16000


In [35]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
8,female,nejhorší situace je v somálsku súdánu mauretán...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, -0.001922607421875, -0.00...",16000
9,female,v letecké i železniční dopravě vize nula platí...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001800537109375, 0.0030517578125, 0.0003356...",16000
10,female,jen tak potvrdíte že před dvaceti lety lidé ne...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, 0.0009765625, -0.00057983...",16000
11,female,v této souvislosti bych očekávala že alespoň d...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000518798828125, -0.0020751953125, -0.0010...",16000
13,female,já sama pocházím z regionu severní moravy ze s...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000457763671875, 0.00201416015625, 0.00283...",16000
...,...,...,...,...,...
1106,female,díky tomu se mnoho informací dostává k adresát...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000244140625, -9.1552734375e-05, 0.0009155...",16000
1109,female,podle dostupných informací turecký premiér dal...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00506591796875, -0.008453369140625, -0.003...",16000
1114,female,veřejné konzultace jsou běžně zveřejňovány pře...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001251220703125, 0.00177001953125, -0.00122...",16000
1117,female,liší se rozsah průvodních dokumentů opět vesmě...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00396728515625, 0.003875732421875, 0.005584...",16000


In [36]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="cs")
    return result["text"]

In [37]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [38]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.5806451612903226
Processed 2 files, WER: 0.6363636363636364
Processed 3 files, WER: 0.7741935483870968
Processed 4 files, WER: 0.6590909090909091
Processed 5 files, WER: 0.782608695652174
Processed 6 files, WER: 0.8181818181818182
Processed 7 files, WER: 0.6538461538461539
Processed 8 files, WER: 0.6363636363636364
Processed 9 files, WER: 0.723404255319149
Processed 10 files, WER: 0.5
Processed 11 files, WER: 1.0
Processed 12 files, WER: 0.5967741935483871
Processed 13 files, WER: 0.9047619047619048
Processed 14 files, WER: 0.6666666666666666
Processed 15 files, WER: 0.625
Processed 16 files, WER: 0.6363636363636364
Processed 17 files, WER: 0.7407407407407407
Processed 18 files, WER: 0.7894736842105263
Processed 19 files, WER: 0.7058823529411765
Processed 20 files, WER: 1.5454545454545454
Processed 21 files, WER: 0.8
Processed 22 files, WER: 0.8571428571428571
Processed 23 files, WER: 1.2857142857142858
Processed 24 files, WER: 0.5416666666666666
Processed 25 

In [39]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.8888888888888888
Processed 2 files, WER: 0.5555555555555556
Processed 3 files, WER: 0.6923076923076923
Processed 4 files, WER: 0.5294117647058824
Processed 5 files, WER: 0.6
Processed 6 files, WER: 0.4444444444444444
Processed 7 files, WER: 0.8666666666666667
Processed 8 files, WER: 0.2857142857142857
Processed 9 files, WER: 0.6285714285714286
Processed 10 files, WER: 0.9230769230769231
Processed 11 files, WER: 0.6666666666666666
Processed 12 files, WER: 0.4444444444444444
Processed 13 files, WER: 0.5555555555555556
Processed 14 files, WER: 0.6666666666666666
Processed 15 files, WER: 0.6304347826086957
Processed 16 files, WER: 0.6388888888888888
Processed 17 files, WER: 1.2307692307692308
Processed 18 files, WER: 0.6486486486486487
Processed 19 files, WER: 0.6470588235294118
Processed 20 files, WER: 0.7931034482758621
Processed 21 files, WER: 0.6
Processed 22 files, WER: 0.6842105263157895
Processed 23 files, WER: 0.6538461538461539
Processed 24 files, WER: 0.